In [1]:
import os
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='6'

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
np.random.seed(2024)
print("data preprocess start")
print("*" * 30)
file_path = '../../../data/jeehoshin/allrecipe/'
# 读取 CSV 文件
raw_recipe = pd.read_csv(file_path + 'raw-data_recipe.csv')
core_recipe = pd.read_csv(file_path + 'core-data_recipe.csv')

data preprocess start
******************************


In [3]:
core_train_rating = pd.read_csv(file_path + 'core-data-train_rating.csv')
core_test_rating = pd.read_csv(file_path + 'core-data-test_rating.csv')
core_valid_rating = pd.read_csv(file_path + 'core-data-valid_rating.csv')

interactions = pd.concat([core_train_rating, core_valid_rating, core_test_rating])

user_ids = interactions['user_id'].unique()
user_to_idx = {uid: i for i, uid in enumerate(user_ids)}
idx_to_user = {i: uid for i, uid in enumerate(user_ids)}

item_ids = interactions['recipe_id'].unique()

item_to_idx = {rid: i for i, rid in enumerate(item_ids)}
idx_to_item = {i: rid for i, rid in enumerate(item_ids)}

print(f"length of user2idx dict: {len(user_to_idx)}")
print(list(user_to_idx.items())[:10])
print(f"length of item2idx dict: {len(item_to_idx)}")
print(list(item_to_idx.items())[:10])

# length of user2idx dict: 68768
# length of item2idx dict: 45630

length of user2idx dict: 68768
[(np.int64(5215572), 0), (np.int64(3622615), 1), (np.int64(1313770), 2), (np.int64(3181149), 3), (np.int64(880574), 4), (np.int64(4053956), 5), (np.int64(4053950), 6), (np.int64(4053953), 7), (np.int64(4203397), 8), (np.int64(1080905), 9)]
length of item2idx dict: 45630
[(np.int64(17991), 0), (np.int64(170724), 1), (np.int64(18045), 2), (np.int64(60598), 3), (np.int64(47519), 4), (np.int64(42964), 5), (np.int64(47822), 6), (np.int64(11815), 7), (np.int64(12780), 8), (np.int64(150711), 9)]


In [4]:
# 시간 순으로 sort하는 것 없앰 -> 다시 사용하는 것으로 바꿈
core_train_rating['u'] = core_train_rating['user_id'].map(user_to_idx)
core_train_rating['i'] = core_train_rating['recipe_id'].map(item_to_idx)
core_train_rating = core_train_rating.sort_values(by=['u', 'dateLastModified'])

core_test_rating['u'] = core_test_rating['user_id'].map(user_to_idx)
core_test_rating['i'] = core_test_rating['recipe_id'].map(item_to_idx)
core_test_rating = core_test_rating.sort_values(by=['u', 'dateLastModified'])

core_valid_rating['u'] = core_valid_rating['user_id'].map(user_to_idx)
core_valid_rating['i'] = core_valid_rating['recipe_id'].map(item_to_idx)
core_valid_rating = core_valid_rating.sort_values(by=['u', 'dateLastModified'])

selected_columns = ['u', 'i', 'rating']
save_train = core_train_rating[selected_columns].copy()
save_valid = core_valid_rating[selected_columns].copy()
save_test = core_test_rating[selected_columns].copy()

In [5]:
save_path = '../../../data/jeehoshin/baseline_dataset_allrecipe_sorted/'
save_train.to_csv(save_path + 'data.train.rating', sep='	', header=False, index=False)
save_valid.to_csv(save_path + 'data.valid.rating', sep='	', header=False, index=False)
save_test.to_csv(save_path + 'data.test.rating', sep='	', header=False, index=False)
print("train/valid/test data saved")

train/valid/test data saved


In [6]:

test_i = save_test['i'].unique()
train_i = save_train['i'].unique()

cold = set(test_i) - set(train_i)
print(len(cold), len(test_i), len(train_i))

# 15707 37342 29093

15707 37342 29093


In [7]:

from collections import Counter, defaultdict
from tqdm import tqdm

ingre_num_df = core_recipe[['recipe_id', 'ingredients']]
ingre_num_df['i'] = ingre_num_df['recipe_id'].map(item_to_idx)
ingre_num_df['ingredient_list'] = ingre_num_df['ingredients'].apply(lambda x: x.split('^'))
ingre_num_df['ingredient_list'] = ingre_num_df['ingredient_list'].apply(lambda x: x[:20])
ingre_num_df = ingre_num_df.sort_values(by=['i'])
train_ingre_df = ingre_num_df[ingre_num_df['i'].isin(train_i)]

df_ing_list = train_ingre_df['ingredient_list'].tolist()
all_ingre_list = [item for sublist in df_ing_list for item in sublist]
frequency_count = Counter(all_ingre_list)
filtered_ingre = set([key for key, value in frequency_count.items() if value >= 1])

ingre_num_df['ingredient_del'] = ingre_num_df['ingredient_list'].apply(
    lambda x: [i for i in x if i in filtered_ingre])
ingre_num_df['ingre_num'] = ingre_num_df['ingredient_del'].apply(lambda x: len(x))

print(ingre_num_df['ingre_num'].value_counts())
print(len(filtered_ingre))

new_ingre = []
for index, row in tqdm(ingre_num_df.iterrows()):
    if row['ingre_num']==0:
        new_ingre.extend(row['ingredient_list'])
    else:
        new_ingre.extend(row['ingredient_del']) 

final_ingre = set(new_ingre)
ingre_num_df['new_ingres'] = ingre_num_df['ingredient_list'].apply(
    lambda x: [i for i in x if i in final_ingre])
ingre_num_df['new_ingres_num'] = ingre_num_df['new_ingres'].apply(lambda x: len(x))

print(len(final_ingre))
print(ingre_num_df['new_ingres_num'].min(), ingre_num_df['new_ingres_num'].max(), ingre_num_df['new_ingres_num'].mean())

from sklearn.preprocessing import LabelEncoder

save_ingre_num = ingre_num_df[['i', 'new_ingres_num']]
save_ingre_num.to_csv(save_path + 'data_id_ingre_num_file', sep='	', header=False, index=False)

ingre_ids = list(final_ingre)
ingre_encoder = LabelEncoder().fit(ingre_ids)

ingre_to_idx = {v: i for i, v in enumerate(ingre_encoder.classes_)}
idx_to_ingre = {i: v for i, v in enumerate(ingre_encoder.classes_)}
ingre_num_df['ingredient_idx'] = ingre_num_df['new_ingres'].apply(lambda x: [ingre_to_idx[i] for i in x])

dict_data = dict(zip(ingre_num_df['i'], ingre_num_df['ingredient_idx'].tolist()))

fill_value = len(ingre_ids)
ingre_code = np.full((len(dict_data), ingre_num_df['new_ingres_num'].max()), fill_value)

for i, (key, value) in enumerate(dict_data.items()):
    ingre_code[key, :len(value)] = value

# save id ingre_code file
# total code num = 33147 
np.save(save_path + 'data_ingre_code_file.npy', ingre_code)

print(ingre_code.shape, np.max(ingre_code))
print(ingre_code, np.max(ingre_code), np.min(ingre_code))
ri_list = []
for i, sublist in tqdm(enumerate(ingre_code)):
    for j in sublist:
        if j != np.max(ingre_code):
            ri_list.append([i, j])
print(len(ri_list))
ri_array = np.array(ri_list).astype(np.int_)
print(ri_array.shape)

ri_path = save_path + 'ri_graph.txt'
with open(ri_path, 'w') as f:
    np.savetxt(f, ri_array, fmt='%d')
print(f"NumPy array saved as {ri_path}")

#ingre_num
#7     4898
#8     4845
#6     4660
#9     4543
#10    4175
#5     3972
#11    3380
#4     2886
#12    2639
#13    2041
#3     1739
#14    1454
#15    1033
#16     826
#2      713
#17     567
#18     379
#20     357
#19     255
#1      207
#0       61
#Name: count, dtype: int64
#19815

#19987
#1 20 8.658470304624151
#(45630, 20) 19987
#[[ 1434  9896  9474 ... 19987 19987 19987]
# [19112  4733  5573 ... 19987 19987 19987]
# [10605 12562 19724 ... 19987 19987 19987]
# ...
# [17493 13506 13740 ... 19987 19987 19987]
# [ 1944 19290 17461 ... 19987 19987 19987]
# [16054  2867 13467 ... 19987 19987 19987]] 19987 0

#395086
#(395086, 2)

/tmp/ipykernel_281231/1438862726.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingre_num_df['i'] = ingre_num_df['recipe_id'].map(item_to_idx)
/tmp/ipykernel_281231/1438862726.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingre_num_df['ingredient_list'] = ingre_num_df['ingredients'].apply(lambda x: x.split('^'))
/tmp/ipykernel_281231/1438862726.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


ingre_num
7     4898
8     4845
6     4660
9     4543
10    4175
5     3972
11    3380
4     2886
12    2639
13    2041
3     1739
14    1454
15    1033
16     826
2      713
17     567
18     379
20     357
19     255
1      207
0       61
Name: count, dtype: int64
19815


45630it [00:01, 25312.45it/s]


19987
1 20 8.658470304624151
(45630, 20) 19987
[[ 9764 16054  9888 ... 19987 19987 19987]
 [ 3638  3971 11553 ... 19987 19987 19987]
 [16975  4733  3661 ... 19987 19987 19987]
 ...
 [19724  3596  7029 ... 19987 19987 19987]
 [19290  5948 19453 ... 19987 19987 19987]
 [ 4229 13439 13506 ... 19987 19987 19987]] 19987 0


45630it [04:38, 163.71it/s]


395086
(395086, 2)
NumPy array saved as ../../../data/jeehoshin/baseline_dataset_allrecipe_sorted/ri_graph.txt


In [8]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

model_name = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name, device=device)

ingre_text_vectors = {}

for key, value in tqdm(ingre_to_idx.items()):
    embedding = model.encode(key, convert_to_numpy=True, show_progress_bar=False)
    ingre_text_vectors[value] = embedding

print(len(ingre_text_vectors))
# 19987

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19987/19987 [02:52<00:00, 115.96it/s]

19987


In [9]:
ingre_feature = np.zeros((len(ingre_text_vectors), ingre_text_vectors[0].shape[0]))

for i, (key, value) in tqdm(enumerate(ingre_text_vectors.items())):
    ingre_feature[key] = value
print(ingre_feature.shape)

np.save(save_path + 'data_ingre_features_allMini.npy', ingre_feature)
#(19987, 384)

19987it [00:00, 409343.75it/s]

(19987, 384)


In [10]:
import re
from collections import defaultdict
from tqdm import tqdm

texture_df = raw_recipe[raw_recipe['recipe_id'].isin(item_to_idx.keys())]
texture_df = texture_df[['recipe_id', 'recipe_name', 'cooking_directions', 'reviews']]

texture_df['i'] = texture_df['recipe_id'].map(item_to_idx)
texture_df = texture_df.sort_values(by=['i'])

title_dict = defaultdict(list)
direction_dict = defaultdict(list)
review_dict = defaultdict(list)

for idx, row in tqdm(texture_df.iterrows()):
    recipe_id = row['i']
    title_dict[recipe_id] += [row['recipe_name']]
    direction = eval(row['cooking_directions'])
    sentences = re.split(r'\n', direction['directions'])

    for sentence in sentences:
        direction_dict[recipe_id] += [sentence]

    reviews = eval(row['reviews'])

    for uid, review in reviews.items():
        review_dict[recipe_id] += [review['text']]

print(len(review_dict))
# 45630

45630it [01:51, 408.19it/s] 

45630


In [11]:
title_vectors = {}

for key, value in tqdm(title_dict.items()):
    embedding = model.encode(value, convert_to_numpy=True, show_progress_bar=False)
    title_vectors[key] = embedding

print(len(title_vectors))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45630/45630 [07:15<00:00, 104.70it/s]

45630


In [12]:
title_feature = np.zeros((len(title_vectors), title_vectors[0].shape[1]))

for i, (key, value) in enumerate(title_vectors.items()):
    title_feature[key] = value[0]

print(title_feature.shape)
np.save(save_path + 'data_title_features_allMini.npy', title_feature)
print(f"save title feature:{title_feature.shape}")

#45630
#(45630, 384)
#save title feature:(45630, 384)

(45630, 384)
save title feature:(45630, 384)


In [13]:

text_feature = np.zeros((len(ingre_code), ingre_feature.shape[1]))
for i, ingre_list in enumerate(ingre_code):
    mask = ingre_list != len(ingre_to_idx)
    new_ingre = ingre_list[mask]
    ingre_embs = ingre_feature[new_ingre]
    ingre_embs = np.concatenate((ingre_embs, np.expand_dims(title_feature[i], axis=0)), axis=0)
    text_feature[i] = ingre_embs.mean(0)
print(text_feature.shape)
np.save(save_path + 'data_text_features_allMini.npy', text_feature)
#(45630, 384)

(45630, 384)


In [14]:
import pickle

filename = save_path + 'mapping_dict.pkl'

# 将字典保存到 pkl 文件中
with open(filename, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump((user_to_idx, item_to_idx, ingre_to_idx), f)
print("save mapping dict")
print(len(ingre_to_idx), len(user_to_idx), len(item_to_idx))

#save mapping dict
#19987 68768 45630

save mapping dict
19987 68768 45630


In [15]:
import torch
from torchvision import models, transforms
from PIL import Image
import os

try:
    from torchvision.models import AlexNet_Weights
    alexnet_weights = AlexNet_Weights.DEFAULT
except Exception:
    alexnet_weights = 'DEFAULT'

caffenet = models.alexnet(weights=alexnet_weights)
# Trim the last classification layer to get a 4096-D embedding
caffenet.classifier = torch.nn.Sequential(*list(caffenet.classifier.children())[:-1])
caffenet.eval()
caffenet.to(device)

image_folder = file_path + 'core-data-images/core-data-images/'

try:
    transform = alexnet_weights.transforms()
except Exception:
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])


# 存储图片向量的字典
image_vectors = {}


num = 0
for index, row in tqdm(ingre_num_df.iterrows()):
    image_id = row['recipe_id']
    image_path = os.path.join(image_folder, f"{image_id}.jpg")
    # 加载并预处理图片
    image = Image.open(image_path)

    try:
        image = transform(image)
        # 增加一个维度，以符合模型的输入要求
        image = image.unsqueeze(0).cuda()
        # 使用模型提取特征
        with torch.no_grad():
            output = caffenet(image)
        # 提取特征向量，并转换为numpy数组
    except Exception as e:
        print(e, image_path)
        image = image.convert('RGB')
        image = transform(image)
        # 增加一个维度，以符合模型的输入要求
        image = image.unsqueeze(0).cuda()
        # 使用模型提取特征
        with torch.no_grad():
            output = caffenet(image)

    vector = output.squeeze().detach().cpu().numpy()
    # 存储特征向量
    image_vectors[image_id] = vector

print(len(image_vectors))

#The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0 /home/zyx2509/Food_dataset/Allrecipes/raw_dataset/core-data-images/core-data-images/165987.jpg
#34802it [04:30, 123.52it/s]
#The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0 /home/zyx2509/Food_dataset/Allrecipes/raw_dataset/core-data-images/core-data-images/229943.jpg
#45630it [06:04, 125.20it/s]
#45630

11622it [01:59, 101.88it/s]

The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0 ../../../data/jeehoshin/allrecipe/core-data-images/core-data-images/165987.jpg


30794it [05:06, 90.43it/s] 

The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0 ../../../data/jeehoshin/allrecipe/core-data-images/core-data-images/229943.jpg


45630it [08:04, 94.10it/s] 

45630


In [16]:
convert_image_vectors = {item_to_idx[key]: value for key, value in image_vectors.items()}

sorted_convert_image_vectors = dict(sorted(convert_image_vectors.items()))
print(sorted_convert_image_vectors[0].shape)

image_feature = np.zeros((len(sorted_convert_image_vectors), sorted_convert_image_vectors[0].size))

# 遍历字典，将值填充到数组中
for i, (key, value) in enumerate(sorted_convert_image_vectors.items()):
    image_feature[key] = value

print(image_feature.shape)
np.save(save_path + 'data_image_features_caffe.npy', image_feature)

#(4096,)
#(45630, 4096)

(4096,)
(45630, 4096)


In [17]:
train_file = save_path + 'data.train.rating'
test_file = save_path + 'data.test.rating'
valid_file = save_path + 'data.valid.rating'

train_df = pd.read_csv(train_file, names=['user', 'item', 'rate'], sep='\t')
test_df = pd.read_csv(test_file, names=['user', 'item', 'rate'], sep='\t')
valid_df = pd.read_csv(valid_file, names=['user', 'item', 'rate'], sep='\t')

from collections import defaultdict
from tqdm import tqdm

item_count = defaultdict(int)
user_items = {}

# 遍历 DataFrame 中的每一行
for index, row in tqdm(train_df.iterrows()):
    # 提取第一列的值作为聚类的键
    u = int(row['user'])
    # 如果键不存在于字典中，则将其初始化为一个空列表
    if u not in user_items:
        user_items[u] = []
    # 将第二列的值添加到对应键的列表中
    user_items[u].append(int(row['item']))
    item_count[int(row['item'])] += 1

print(len(item_count))

all_item = list(item_count.keys())
count = list(item_count.values())
sum_value = np.sum([x for x in count])
popularity = [value / sum_value for value in count]

sum_prob = np.sum([x ** 0.7 for x in popularity])
probability = [value ** 0.7 / sum_prob for value in popularity]

np.random.seed(2024)
item_set = list(item_count.keys())
test_num = 500
sample_type = 'popular bias'
test_neg_items = defaultdict()
valid_neg_items = defaultdict()
valid_user_set = set(valid_df['user'])
test_user_set = set(test_df['user'])

for user, user_seq in tqdm(user_items.items()):
    if user not in test_user_set:
        continue
    test_samples = []
    while len(test_samples) < test_num:
        sample_ids = np.random.choice(all_item, 2 * test_num, replace=False, p=probability)
        sample_ids_set = set(sample_ids)
        user_seq_set = set(user_seq)
        result_set = sample_ids_set - user_seq_set
        test_samples = list(result_set)
    test_samples = test_samples[:test_num]
    test_neg_items[user] = test_samples

for user, user_seq in tqdm(user_items.items()):
    if user not in valid_user_set:
        continue
    valid_samples = []
    while len(valid_samples) < test_num:
        sample_ids = np.random.choice(all_item, 2 * test_num, replace=False, p=probability)
        sample_ids_set = set(sample_ids)
        user_seq_set = set(user_seq)
        result_set = sample_ids_set - user_seq_set
        valid_samples = list(result_set)
    valid_samples = valid_samples[:test_num]
    valid_neg_items[user] = valid_samples

# 29093

676946it [00:23, 29080.80it/s]


29093


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68768/68768 [02:58<00:00, 385.40it/s]


In [18]:
test_pos_dict = {}
valid_pos_dict = {}

for index, row in tqdm(valid_df.iterrows()):
    # 提取第一列的值作为聚类的键
    u = int(row['user'])
    if u not in valid_pos_dict:
        valid_pos_dict[u] = []
    valid_pos_dict[u].append(int(row['item']))

for index, row in tqdm(test_df.iterrows()):
    # 提取第一列的值作为聚类的键
    u = int(row['user'])
    if u not in test_pos_dict:
        test_pos_dict[u] = []
    test_pos_dict[u].append(int(row['item']))

test_sample_pos = pd.DataFrame({'KeyValue': [f"{k}:{v}" for k, v in test_pos_dict.items()]}, index=test_pos_dict.keys())
valid_sample_pos = pd.DataFrame({'KeyValue': [f"{k}:{v}" for k, v in valid_pos_dict.items()]},
                                index=valid_pos_dict.keys())

print(test_sample_pos[:5])
print(valid_sample_pos[:5])
#                                            KeyValue
#0                       0:[17121, 4292, 8417, 31007]
#1  1:[13687, 9409, 10495, 18825, 33844, 3123, 398...
#2  2:[8882, 7784, 21682, 21666, 30040, 13348, 375...
#3                              3:[1203, 1444, 17970]
#4  4:[35035, 32289, 25895, 20344, 26340, 20173, 3...
#                                             KeyValue
#1                                            1:[3914]
#2   2:[32581, 31299, 7767, 32286, 31378, 5117, 314...
#3                                    3:[19092, 12027]
#4   4:[5024, 5696, 25525, 8333, 5803, 4300, 8924, ...
#10                            10:[31007, 12507, 2607]

133459it [00:04, 32062.12it/s]
283440it [00:08, 32153.89it/s]


                                            KeyValue
0                              0:[15805, 24518, 877]
1                                            1:[877]
2                         2:[4882, 3559, 3667, 5641]
3                                     3:[13444, 872]
4  4:[8783, 15373, 16240, 29242, 29736, 1759, 361...
                                             KeyValue
2                                            2:[2159]
4              4:[13059, 1671, 3278, 1167, 865, 1549]
6   6:[26954, 6886, 5605, 6229, 10028, 14632, 5198...
7    7:[877, 2710, 1548, 1346, 175, 1360, 4321, 4622]
10                                          10:[9732]


In [19]:

test_sample_neg = pd.DataFrame.from_dict(test_neg_items).transpose()
valid_sample_neg = pd.DataFrame.from_dict(valid_neg_items).transpose()

print(len(test_sample_neg))
print(len(valid_sample_neg))

#68768
#29303

68768
29303


In [20]:
test_sample_concat = pd.concat([test_sample_pos, test_sample_neg], axis=1)
print(len(test_sample_concat))

valid_sample_concat = pd.concat([valid_sample_pos, valid_sample_neg], axis=1)
print(len(valid_sample_concat))

#68768
#29303

68768
29303


In [21]:
test_sample_concat['KeyValue'] = test_sample_concat['KeyValue'].apply(
    lambda x: x.replace('[', '').replace(']', '').replace(' ', ''))
test_sample_concat['KeyValue'] = test_sample_concat['KeyValue'].apply(lambda x: f"({x})")

print(test_sample_concat.iloc[:5, :5])

valid_sample_concat['KeyValue'] = valid_sample_concat['KeyValue'].apply(
    lambda x: x.replace('[', '').replace(']', '').replace(' ', ''))
valid_sample_concat['KeyValue'] = valid_sample_concat['KeyValue'].apply(lambda x: f"({x})")

print(valid_sample_concat.iloc[:5, :5])

#KeyValue      0      1      2  \
#0                          (0:17121,4292,8417,31007)   6144      3  16388   
#1  (1:13687,9409,10495,18825,33844,3123,3981,4132...  16384  18433  16386   
#2  (2:8882,7784,21682,21666,30040,13348,37579,195...  12288   2049  20482   
#3                                (3:1203,1444,17970)  18432  14337   6146   
#4  (4:35035,32289,25895,20344,26340,20173,32105,3...  12290   2051  20484   

#       3  
#0  10244  
#1  28675  
#2  20483  
#3  20483  
#4   6148  
#                                             KeyValue      0      1      2  \
#1                                            (1:3914)      2   6154  28682   
#2   (2:32581,31299,7767,32286,31378,5117,31418,900...  22528  18432      3   
#3                                     (3:19092,12027)  20481   6146  24579   
#4   (4:5024,5696,25525,8333,5803,4300,8924,24105,2...  20484  12293  30728   
#10                              (10:31007,12507,2607)   4097  16386  12294   

#        3  
#1    8203  
#2   28676  
#3   20483  
#4   12300  
#10   4105  

                                            KeyValue      0      1      2  \
0                                (0:15805,24518,877)   4096  12288  12290   
1                                            (1:877)   8200   4105  16397   
2                            (2:4882,3559,3667,5641)  16387  10243   2053   
3                                      (3:13444,872)      0   6147  10244   
4  (4:8783,15373,16240,29242,29736,1759,3612,3058...      0      2  22531   

      3  
0  4099  
1    15  
2    10  
3     6  
4  2053  
                                             KeyValue      0      1      2  \
2                                            (2:2159)   4096   4101   4103   
4                   (4:13059,1671,3278,1167,865,1549)      7   2055  16393   
6   (6:26954,6886,5605,6229,10028,14632,5198,23598...  10241      2   8196   
7           (7:877,2710,1548,1346,175,1360,4321,4622)      0  10241  22530   
10                                          (10:9732)   2049      4  20485   

        

In [22]:
valid_sample_concat.to_csv(save_path + 'data.valid.negative', sep='	', header=False, index=False)
test_sample_concat.to_csv(save_path + 'data.test.negative', sep='	', header=False, index=False)

In [23]:
from scipy.sparse import coo_matrix

src = save_train['u'].values
tgt = save_train['i'].values
data = np.ones(len(save_train))
mat = coo_matrix((data, (src, tgt)), shape=(len(user_to_idx), len(item_to_idx)))

# 指定保存到的文件名
import pickle

filename = save_path + 'inter_coo_matrix.pkl'

# 将字典保存到 pkl 文件中
with open(filename, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump(mat, f)
print("save training set sparse coo matrix")

save training set sparse coo matrix


In [24]:
# now, begin process graph edges
# 1 user-recipe graph edge
import numpy as np

inter_mat = []
lines = open(save_path + 'data.train.rating', 'r').readlines()
for line in lines:
    tokens = line.strip().split('\t')
    u_id, pos_id = int(tokens[0]), int(tokens[1])
    inter_mat.append([u_id, pos_id])

ur_array = np.array(inter_mat).astype(np.int_)

print(ur_array.shape)

ur_path = save_path + 'graph_edge/ur_graph.txt'
with open(ur_path, 'w') as f:
    np.savetxt(f, ur_array, fmt='%d')
print(f"NumPy array saved as {ur_path}")

with open(ur_path, 'r') as f:
    ab = np.loadtxt(f, dtype=np.int_)
print(f"testing for shape, ur_graph is {ab.shape}")

#(676946, 2)
#NumPy array saved as /home/zyx2509/Food_dataset/Allrecipes/processed_dataset/graph_edge/ur_graph.txt
#testing for shape, ur_graph is (676946, 2)

(676946, 2)
NumPy array saved as ../../../data/jeehoshin/baseline_dataset_allrecipe_sorted/graph_edge/ur_graph.txt
testing for shape, ur_graph is (676946, 2)


In [25]:
# 2 recipe-recipe graph edge 
# 2 pending
# 3 recipe-ingredient graph edge
from tqdm import tqdm

ingre_code = np.load(save_path + 'data_ingre_code_file.npy')
print(ingre_code, np.max(ingre_code), np.min(ingre_code))
ri_list = []
for i, sublist in tqdm(enumerate(ingre_code)):
    for j in sublist:
        if j != np.max(ingre_code):
            ri_list.append([i, j])
print(len(ri_list))
ri_array = np.array(ri_list).astype(np.int_)
print(ri_array.shape)

ri_path = save_path + 'graph_edge/ri_graph.txt'
with open(ri_path, 'w') as f:
    np.savetxt(f, ri_array, fmt='%d')
print(f"NumPy array saved as {ri_path}")

with open(ri_path, 'r') as f:
    ab = np.loadtxt(f, dtype=np.int_)
print(f"testing for shape, ri_graph is {ab.shape}")

#[[ 1434  9896  9474 ... 19987 19987 19987]
# [19112  4733  5573 ... 19987 19987 19987]
# [10605 12562 19724 ... 19987 19987 19987]
# ...
# [17493 13506 13740 ... 19987 19987 19987]
# [ 1944 19290 17461 ... 19987 19987 19987]
# [16054  2867 13467 ... 19987 19987 19987]] 19987 0
#395086
#(395086, 2)
#NumPy array saved as /home/zyx2509/Food_dataset/Allrecipes/processed_dataset/graph_edge/ri_graph.txt
#testing for shape, ri_graph is (395086, 2)

[[ 9764 16054  9888 ... 19987 19987 19987]
 [ 3638  3971 11553 ... 19987 19987 19987]
 [16975  4733  3661 ... 19987 19987 19987]
 ...
 [19724  3596  7029 ... 19987 19987 19987]
 [19290  5948 19453 ... 19987 19987 19987]
 [ 4229 13439 13506 ... 19987 19987 19987]] 19987 0


45630it [05:03, 150.10it/s]


395086
(395086, 2)
NumPy array saved as ../../../data/jeehoshin/baseline_dataset_allrecipe_sorted/graph_edge/ri_graph.txt
testing for shape, ri_graph is (395086, 2)


In [26]:
# 4 ingredient-ingredient graph edge
# 4 used for fgcn model
from collections import Counter

mapping_dict = save_path + 'mapping_dict.pkl'

with open(mapping_dict, 'rb') as f:
    u_to_idx, i_to_idx, ingre_to_idx = pickle.load(f)
print(len(ingre_to_idx))

color_set = ['white', 'black', 'red', 'green', 'yellow']
ingre_with_colors = defaultdict(list)

# 遍历字典中的每个值
for key, value in tqdm(ingre_to_idx.items()):
    # 检查值中是否包含颜色列表中的任何一个颜色
    for color in color_set:
        if color in key:
            # 如果包含颜色，则将对应的键保存到列表中
            ingre_with_colors[color].append(value)

shape_set = ['slice', 'dice', 'minced', 'powder', 'roll', 'shred']
ingre_with_shapes = defaultdict(list)

# 遍历字典中的每个值
for key, value in tqdm(ingre_to_idx.items()):
    # 检查值中是否包含颜色列表中的任何一个颜色
    for shape in shape_set:
        if shape in key:
            # 如果包含颜色，则将对应的键保存到列表中
            ingre_with_shapes[shape].append(value)

cooking_set = ['deep-fry', 'dry', 'fry', 'steam', 'boil', 'pickle']
ingre_with_cookings = defaultdict(list)

# 遍历字典中的每个值
for key, value in tqdm(ingre_to_idx.items()):
    # 检查值中是否包含颜色列表中的任何一个颜色
    for cooking in cooking_set:
        if cooking in key:
            # 如果包含颜色，则将对应的键保存到列表中
            ingre_with_cookings[cooking].append(value)

edge_dict = defaultdict(int)

for key, lst in tqdm(ingre_with_colors.items()):
    for i in range(len(lst)):
        for j in range(i + 1, len(lst)):
            edge_dict[(lst[i], lst[j])] += 1
for key, lst in tqdm(ingre_with_shapes.items()):
    for i in range(len(lst)):
        for j in range(i + 1, len(lst)):
            edge_dict[(lst[i], lst[j])] += 1
for key, lst in tqdm(ingre_with_cookings.items()):
    for i in range(len(lst)):
        for j in range(i + 1, len(lst)):
            edge_dict[(lst[i], lst[j])] += 1
print(len(edge_dict))
value_counts = Counter(val for val in edge_dict.values())

# 打印结果
print("每个元素出现的次数：", value_counts)

import random

random.seed(2024)

new_edge_list = []
for edge_pair, count in tqdm(edge_dict.items()):
    if count == 1:
        random_number = random.random()
        if random_number < 0.025:
            new_edge_list.append([edge_pair[0], edge_pair[1]])
            new_edge_list.append([edge_pair[1], edge_pair[0]])
    else:
        new_edge_list.append([edge_pair[0], edge_pair[1]])
        new_edge_list.append([edge_pair[1], edge_pair[0]])

print(len(new_edge_list))
print(new_edge_list[:10])

ii_array = np.array(new_edge_list).astype(np.int_)

print(ii_array.shape)

ii_path = save_path + 'graph_edge/ii_graph.txt'
with open(ii_path, 'w') as f:
    np.savetxt(f, ii_array, fmt='%d')
print(f"NumPy array saved as {ii_path}")

with open(ii_path, 'r') as f:
    ab = np.loadtxt(f, dtype=np.int_)
print(ab.shape)

# 19987
# 3205621
# 292488
# [[38, 744], [744, 38], [38, 945], [945, 38], [38, 2107], [2107, 38], [38, 2913], [2913, 38], [38, 4341], [4341, 38]]
# (292488, 2)
# NumPy array saved as /home/zyx2509/Food_dataset/Allrecipes/processed_dataset/graph_edge/ii_graph.txt
# (292488, 2)

19987


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 160.39it/s]


3205621
每个元素出现的次数： Counter({1: 3138051, 2: 67436, 3: 134})


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3205621/3205621 [00:03<00:00, 805487.75it/s]


292488
[[38, 744], [744, 38], [38, 945], [945, 38], [38, 2107], [2107, 38], [38, 2913], [2913, 38], [38, 4341], [4341, 38]]
(292488, 2)
NumPy array saved as ../../../data/jeehoshin/baseline_dataset_allrecipe_sorted/graph_edge/ii_graph.txt
(292488, 2)


In [27]:
# 5 recipe-calories graph edge
# 7 calories level dict
# 5 and 7 used for schgn model
# build calories dict for 5 and 7
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder

calories_dict = {}
raw_recipe = pd.read_csv(file_path + 'raw-data_recipe.csv')
with open(save_path + 'mapping_dict.pkl', 'rb') as f:
    _, item_to_idx, _ = pickle.load(f)

items = item_to_idx.keys()
print(len(items))
now_recipe = raw_recipe[raw_recipe['recipe_id'].isin(items)]
now_recipe['cal'] = now_recipe['nutritions'].apply(lambda x: int(eval(x)[u'calories'][u'amount'] / 30))
now_recipe['i'] = now_recipe['recipe_id'].apply(lambda x: item_to_idx[x])
print(now_recipe['cal'].max(), now_recipe['cal'].min(), now_recipe['cal'].mean())

cal_encoder = LabelEncoder().fit(list(now_recipe['cal'].unique()))
cal_to_idx = {v: i for i, v in enumerate(cal_encoder.classes_)}
print(len(cal_to_idx))

now_recipe['cal_norm'] = now_recipe['cal'].apply(lambda x: cal_to_idx[x])
now_recipe = now_recipe.sort_values(by='i')

for index, row in now_recipe.iterrows():
    recipe_id = row['i']
    calories_amount = row['cal_norm']
    calories_dict[recipe_id] = calories_amount
# 5 recipe-calories graph edge
rc_list = []
for key, value in calories_dict.items():
    rc_list.append([key, value])
rc_array = np.array(rc_list).astype(np.int_)
print(rc_array.shape)

rc_path = save_path + 'graph_edge/rc_graph.txt'
with open(rc_path, 'w') as f:
    np.savetxt(f, rc_array, fmt='%d')
print("NumPy array saved as 'rc_graph.txt'")

with open(rc_path, 'r') as f:
    ab = np.loadtxt(f, dtype=np.int_)
print(f"testing for shape, rc_graph is {ab.shape}")

# 7 calories level dict
rc_dict = save_path + 'graph_edge/recipe_cal_level_dict.pkl'

# 将字典保存到 pkl 文件中
with open(rc_dict, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump(calories_dict, f)
print("save calories dict")

cal_to_idx_map_file = save_path + 'graph_edge/recipe_cal_level_map.pkl'
with open(cal_to_idx_map_file, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump(cal_to_idx, f)
print("save cal_to_idx dict")
# 45630
# 156 0 10.235130396668858
#80
#(45630, 2)
#NumPy array saved as 'rc_graph.txt'
#testing for shape, rc_graph is (45630, 2)
#save calories dict
#save cal_to_idx dict

45630


/tmp/ipykernel_281231/4112088140.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  now_recipe['cal'] = now_recipe['nutritions'].apply(lambda x: int(eval(x)[u'calories'][u'amount'] / 30))
/tmp/ipykernel_281231/4112088140.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  now_recipe['i'] = now_recipe['recipe_id'].apply(lambda x: item_to_idx[x])
/tmp/ipykernel_281231/4112088140.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

156 0 10.235130396668858
80
(45630, 2)
NumPy array saved as 'rc_graph.txt'
testing for shape, rc_graph is (45630, 2)
save calories dict
save cal_to_idx dict


In [28]:
# begin to calculate health level
import pandas as pd
import pickle
from tqdm import tqdm

# 将字典保存到 pkl 文件中
raw_recipe = pd.read_csv(file_path + 'core-data_recipe.csv')
with open(save_path + 'mapping_dict.pkl', 'rb') as f:
    _, item_to_idx, _ = pickle.load(f)
dataset_recipe = raw_recipe[raw_recipe['recipe_id'].isin(list(item_to_idx.keys()))]
recipe_nutrition = dataset_recipe[['recipe_id', 'recipe_name', 'nutritions']]


def extract_num(text):
    print(text)
    numbers = re.findall(r'\d+', text)
    # 如果有多个数字，将它们连接成一个字符串
    result = ''.join(numbers)
    # 将字符串转换为整数
    result_int = int(result)
    return result_int


for idx, row in tqdm(recipe_nutrition.iterrows()):
    calories_amount = eval(row['nutritions'])[u'calories'][u'amount']
    fat_pdv = eval(row['nutritions'])[u'fat'][u'percentDailyValue']
    sugar_pdv = eval(row['nutritions'])[u'sugars'][u'amount']
    sodium_pdv = eval(row['nutritions'])[u'sodium'][u'amount']
    protein_pdv = eval(row['nutritions'])[u'protein'][u'percentDailyValue']
    saturated_fat_pdv = eval(row['nutritions'])[u'saturatedFat'][u'percentDailyValue']
    carbohydrates_pdv = eval(row['nutritions'])[u'carbohydrates'][u'percentDailyValue']
    fiber = eval(row['nutritions'])[u'fiber'][u'amount']
    recipe_nutrition.at[idx, 'cal'] = calories_amount
    recipe_nutrition.at[idx, 'fat'] = fat_pdv
    recipe_nutrition.at[idx, 'sugar'] = sugar_pdv
    recipe_nutrition.at[idx, 'sodium'] = sodium_pdv
    recipe_nutrition.at[idx, 'protein'] = protein_pdv
    recipe_nutrition.at[idx, 'saturated_fat'] = saturated_fat_pdv
    recipe_nutrition.at[idx, 'carbohydrates'] = carbohydrates_pdv
    recipe_nutrition.at[idx, 'fiber'] = fiber

recipe_nutrition

0it [00:00, ?it/s]/tmp/ipykernel_281231/2654113252.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe_nutrition.at[idx, 'cal'] = calories_amount
/tmp/ipykernel_281231/2654113252.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe_nutrition.at[idx, 'fat'] = fat_pdv
45630it [02:53, 263.50it/s]


,recipe_id,recipe_name,nutritions,cal,fat,sugar,sodium,protein,saturated_fat,carbohydrates,fiber
0,240488,"Pork Loin, Apples, and Sauerkraut","{u'niacin': {u'hasCompleteData': False, u'name...",371.72190,18,19.841460,2606.76400,73,18,10,10.223360
1,218939,Foolproof Rosemary Chicken Wings,"{u'niacin': {u'hasCompleteData': True, u'name'...",335.16550,36,0.236130,762.80500,48,28,2,0.886681
2,87211,Chicken Pesto Paninis,"{u'niacin': {u'hasCompleteData': True, u'name'...",640.56170,45,1.974210,1075.52700,65,55,20,4.441335
3,245714,Potato Bacon Pizza,"{u'niacin': {u'hasCompleteData': True, u'name'...",162.66850,12,2.452803,189.80880,14,20,5,0.749092
4,218545,Latin-Inspired Spicy Cream Chicken Stew,"{u'niacin': {u'hasCompleteData': False, u'name...",334.00750,12,8.222667,1301.88800,52,19,12,8.019139
...,...,...,...,...,...,...,...,...,...,...,...
45625,222886,Grateful Dead Cocktail,"{u'niacin': {u'hasCompleteData': False, u'name...",403.19880,< 1,18.099000,10.77593,< 1,< 1,6,0.000000
45626,25650,Cheese Filling For Pastries,"{u'niacin': {u'hasCompleteData': True, u'name'...",115.52360,14,5.549170,70.75883,4,27,2,0.097012
45627,23544,Peach Smoothie,"{u'niacin': {u'hasCompleteData': False, u'name...",151.52430,7,20.284320,74.95521,10,8,8,2.010828
45628,170710,Double Dare Peaches,"{u'niacin': {u'hasCompleteData': True, u'name'...",402.52910,33,44.335170,183.89440,11,65,16,1.303431


In [29]:
recipe_nutrition['fat'] = recipe_nutrition['fat'].replace('< 1', '0')
col = pd.to_numeric(recipe_nutrition['fat'], errors='coerce')  # 숫자로 변환, 안 되면 NaN
recipe_nutrition['fat'] = col.fillna(col.median())
#recipe_nutrition['fat'] = recipe_nutrition['fat'].fillna(int(recipe_nutrition['fat'].median()))
recipe_nutrition['fat'] = recipe_nutrition['fat'].astype(int)

recipe_nutrition['protein'] = recipe_nutrition['protein'].replace('< 1', '0')
col = pd.to_numeric(recipe_nutrition['protein'], errors='coerce')
recipe_nutrition['protein'] = col.fillna(col.median())
#recipe_nutrition['protein'] = recipe_nutrition['protein'].fillna(int(recipe_nutrition['protein'].median()))
recipe_nutrition['protein'] = recipe_nutrition['protein'].astype(int)

recipe_nutrition['saturated_fat'] = recipe_nutrition['saturated_fat'].replace('< 1', '0')
col = pd.to_numeric(recipe_nutrition['saturated_fat'], errors='coerce')
recipe_nutrition['saturated_fat'] = col.fillna(col.median())
#recipe_nutrition['saturated_fat'] = recipe_nutrition['saturated_fat'].fillna(int(recipe_nutrition['saturated_fat'].median()))
recipe_nutrition['saturated_fat'] = recipe_nutrition['saturated_fat'].astype(int)

recipe_nutrition['carbohydrates'] = recipe_nutrition['carbohydrates'].replace('< 1', '0')
col = pd.to_numeric(recipe_nutrition['carbohydrates'], errors='coerce')
recipe_nutrition['carbohydrates'] = col.fillna(col.median())
#recipe_nutrition['carbohydrates'] = recipe_nutrition['carbohydrates'].fillna(int(recipe_nutrition['carbohydrates'].median()))
recipe_nutrition['carbohydrates'] = recipe_nutrition['carbohydrates'].astype(int)
recipe_nutrition

,recipe_id,recipe_name,nutritions,cal,fat,sugar,sodium,protein,saturated_fat,carbohydrates,fiber
0,240488,"Pork Loin, Apples, and Sauerkraut","{u'niacin': {u'hasCompleteData': False, u'name...",371.72190,18,19.841460,2606.76400,73,18,10,10.223360
1,218939,Foolproof Rosemary Chicken Wings,"{u'niacin': {u'hasCompleteData': True, u'name'...",335.16550,36,0.236130,762.80500,48,28,2,0.886681
2,87211,Chicken Pesto Paninis,"{u'niacin': {u'hasCompleteData': True, u'name'...",640.56170,45,1.974210,1075.52700,65,55,20,4.441335
3,245714,Potato Bacon Pizza,"{u'niacin': {u'hasCompleteData': True, u'name'...",162.66850,12,2.452803,189.80880,14,20,5,0.749092
4,218545,Latin-Inspired Spicy Cream Chicken Stew,"{u'niacin': {u'hasCompleteData': False, u'name...",334.00750,12,8.222667,1301.88800,52,19,12,8.019139
...,...,...,...,...,...,...,...,...,...,...,...
45625,222886,Grateful Dead Cocktail,"{u'niacin': {u'hasCompleteData': False, u'name...",403.19880,0,18.099000,10.77593,0,0,6,0.000000
45626,25650,Cheese Filling For Pastries,"{u'niacin': {u'hasCompleteData': True, u'name'...",115.52360,14,5.549170,70.75883,4,27,2,0.097012
45627,23544,Peach Smoothie,"{u'niacin': {u'hasCompleteData': False, u'name...",151.52430,7,20.284320,74.95521,10,8,8,2.010828
45628,170710,Double Dare Peaches,"{u'niacin': {u'hasCompleteData': True, u'name'...",402.52910,33,44.335170,183.89440,11,65,16,1.303431


In [30]:
recipe_nutrition['i'] = recipe_nutrition['recipe_id'].apply(lambda x: item_to_idx[x])

recipe_nutrition['n1'] = recipe_nutrition['fat'].apply(lambda x: 1 if 15 <= x <= 30 else 0)
recipe_nutrition['n2'] = recipe_nutrition['sugar'].apply(lambda x: 1 if x < 5 else 0)
recipe_nutrition['n3'] = recipe_nutrition['sodium'].apply(lambda x: 1 if x < 2000 else 0)
recipe_nutrition['n4'] = recipe_nutrition['protein'].apply(lambda x: 1 if 10 <= x <= 15 else 0)
recipe_nutrition['n5'] = recipe_nutrition['saturated_fat'].apply(lambda x: 1 if x < 10 else 0)
recipe_nutrition['n6'] = recipe_nutrition['carbohydrates'].apply(lambda x: 1 if 55 <= x <= 75 else 0)
recipe_nutrition['n7'] = recipe_nutrition['fiber'].apply(lambda x: 1 if x > 10 else 0)
recipe_nutrition = recipe_nutrition.sort_values(by='i').reset_index(drop=True)
numeric_columns = recipe_nutrition.iloc[:, -7:].values.tolist()
list_df = pd.DataFrame({'nut':numeric_columns})
concat = pd.concat([recipe_nutrition, list_df], axis = 1)
print(concat['nut'].value_counts())

multi_hot_health_dict = {}
for index, row in concat.iterrows():
    recipe_id = row['i']
    health_list = row['nut']
    multi_hot_health_dict[recipe_id] = health_list
rh_dict_mh = save_path + 'graph_edge/recipe_health_level_multi_hot_dict.pkl'

# 将字典保存到 pkl 文件中
with open(rh_dict_mh, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump(multi_hot_health_dict, f)
print("save health dict")
concat

#[0, 0, 1, 0, 0, 0, 0]    8642
#[0, 1, 1, 0, 0, 0, 0]    7334
#[1, 0, 1, 0, 0, 0, 0]    6068
#[1, 1, 1, 0, 0, 0, 0]    5468
#[0, 0, 1, 0, 1, 0, 0]    4823
#                         ... 
#[0, 0, 0, 0, 1, 1, 1]       1
#[0, 0, 1, 0, 0, 1, 1]       1
#[1, 1, 0, 1, 0, 0, 0]       1
#[1, 0, 0, 1, 0, 0, 0]       1
#[1, 0, 1, 0, 0, 1, 1]       1

nut
[0, 0, 1, 0, 0, 0, 0]    8642
[0, 1, 1, 0, 0, 0, 0]    7334
[1, 0, 1, 0, 0, 0, 0]    6068
[1, 1, 1, 0, 0, 0, 0]    5468
[0, 0, 1, 0, 1, 0, 0]    4823
                         ... 
[0, 0, 0, 0, 1, 1, 1]       1
[0, 1, 1, 1, 1, 0, 1]       1
[0, 1, 0, 0, 0, 1, 0]       1
[0, 0, 0, 1, 0, 0, 1]       1
[1, 0, 1, 0, 0, 1, 1]       1
Name: count, Length: 64, dtype: int64
save health dict


,recipe_id,recipe_name,nutritions,cal,fat,sugar,sodium,protein,saturated_fat,carbohydrates,fiber,i,n1,n2,n3,n4,n5,n6,n7,nut
0,17991,Stuffed Green Peppers I,"{u'niacin': {u'hasCompleteData': True, u'name'...",462.82970,37,12.233880,970.63610,54,62,11,3.457861,0,0,0,1,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0]"
1,170724,Sean's Mommy's Roasted Root Vegetables,"{u'niacin': {u'hasCompleteData': False, u'name...",210.19400,9,11.239960,121.31600,7,4,13,8.148600,1,0,0,1,0,1,0,0,"[0, 0, 1, 0, 1, 0, 0]"
2,18045,Yellow Squash Casserole,"{u'niacin': {u'hasCompleteData': True, u'name'...",195.67160,23,1.681259,463.48540,12,39,3,1.300000,2,1,1,1,1,0,0,0,"[1, 1, 1, 1, 0, 0, 0]"
3,60598,Vegetarian Korma,"{u'niacin': {u'hasCompleteData': True, u'name'...",461.67320,48,9.293081,1433.91800,17,76,13,8.414915,3,0,0,1,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0]"
4,47519,Cream Cheese Chicken,"{u'niacin': {u'hasCompleteData': False, u'name...",468.49650,29,36.508200,677.74930,64,38,14,1.043333,4,1,0,1,0,0,0,0,"[1, 0, 1, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45625,228837,Fruit Pizza Even Better,"{u'niacin': {u'hasCompleteData': False, u'name...",290.15710,26,18.069380,124.36180,6,52,11,1.238469,45625,1,0,1,0,0,0,0,"[1, 0, 1, 0, 0, 0, 0]"
45626,244930,Skinny Girl Individual Omelets,"{u'niacin': {u'hasCompleteData': False, u'name...",122.85000,13,0.796500,223.15520,21,23,0,0.897417,45626,0,1,1,0,0,0,0,"[0, 1, 1, 0, 0, 0, 0]"
45627,238639,Chai Snickerdoodle Cookies,"{u'niacin': {u'hasCompleteData': False, u'name...",191.00790,13,16.216050,137.30190,4,25,9,0.590269,45627,0,0,1,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0]"
45628,234369,Baghdad Beef Stew,"{u'niacin': {u'hasCompleteData': False, u'name...",684.93640,67,36.653840,677.42070,60,54,16,9.197719,45628,0,0,1,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0]"


In [31]:
for idx, row in tqdm(recipe_nutrition.iterrows()):
    health_level = 0
    # 65g
    if 15 <= row['fat'] <= 30:
        health_level += 1
    # WHO healthy sugars 50g per day, and PDV <10% is health, 5g for amount 
    if row['sugar'] < 5:
        health_level += 1
    # WHO healthy sodium 2g(2000mg) per day, daily 2400mg, 2/2.4 = 83 for percent
    # WHO healthy salt 5g per day, daily 6000mg
    if row['sodium'] < 2000:
        health_level += 1
    # 50g
    if 10 <= row['protein'] <= 15:
        health_level += 1
    # 20g
    if row['saturated_fat'] < 10:
        health_level += 1
        # 300g
    if 55 <= row['carbohydrates'] <= 75:
        health_level += 1
    # >25g
    if row['fiber'] > 10:
        health_level += 1
    recipe_nutrition.at[idx, 'health'] = health_level
recipe_nutrition

45630it [00:03, 13028.90it/s]


,recipe_id,recipe_name,nutritions,cal,fat,sugar,sodium,protein,saturated_fat,carbohydrates,fiber,i,n1,n2,n3,n4,n5,n6,n7,health
0,17991,Stuffed Green Peppers I,"{u'niacin': {u'hasCompleteData': True, u'name'...",462.82970,37,12.233880,970.63610,54,62,11,3.457861,0,0,0,1,0,0,0,0,1.0
1,170724,Sean's Mommy's Roasted Root Vegetables,"{u'niacin': {u'hasCompleteData': False, u'name...",210.19400,9,11.239960,121.31600,7,4,13,8.148600,1,0,0,1,0,1,0,0,2.0
2,18045,Yellow Squash Casserole,"{u'niacin': {u'hasCompleteData': True, u'name'...",195.67160,23,1.681259,463.48540,12,39,3,1.300000,2,1,1,1,1,0,0,0,4.0
3,60598,Vegetarian Korma,"{u'niacin': {u'hasCompleteData': True, u'name'...",461.67320,48,9.293081,1433.91800,17,76,13,8.414915,3,0,0,1,0,0,0,0,1.0
4,47519,Cream Cheese Chicken,"{u'niacin': {u'hasCompleteData': False, u'name...",468.49650,29,36.508200,677.74930,64,38,14,1.043333,4,1,0,1,0,0,0,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45625,228837,Fruit Pizza Even Better,"{u'niacin': {u'hasCompleteData': False, u'name...",290.15710,26,18.069380,124.36180,6,52,11,1.238469,45625,1,0,1,0,0,0,0,2.0
45626,244930,Skinny Girl Individual Omelets,"{u'niacin': {u'hasCompleteData': False, u'name...",122.85000,13,0.796500,223.15520,21,23,0,0.897417,45626,0,1,1,0,0,0,0,2.0
45627,238639,Chai Snickerdoodle Cookies,"{u'niacin': {u'hasCompleteData': False, u'name...",191.00790,13,16.216050,137.30190,4,25,9,0.590269,45627,0,0,1,0,0,0,0,1.0
45628,234369,Baghdad Beef Stew,"{u'niacin': {u'hasCompleteData': False, u'name...",684.93640,67,36.653840,677.42070,60,54,16,9.197719,45628,0,0,1,0,0,0,0,1.0


In [32]:
recipe_nutrition['i'] = recipe_nutrition['recipe_id'].apply(lambda x: item_to_idx[x])
print(recipe_nutrition['health'].value_counts())

health_dict = {}
for index, row in recipe_nutrition.iterrows():
    recipe_id = row['i']
    health_amount = row['health']
    health_dict[recipe_id] = int(health_amount)
rh_dict = save_path + 'graph_edge/recipe_health_level_dict.pkl'

# 将字典保存到 pkl 文件中
with open(rh_dict, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump(health_dict, f)
print("save health dict")

import numpy as np
# recipe-health graph edge
rh_list = []
for key, value in health_dict.items():
    rh_list.append([key, value])
rh_array = np.array(rh_list).astype(np.int_)

print(rh_array.shape)

rh_path = save_path + 'graph_edge/rh_graph.txt'
with open(rh_path, 'w') as f:
    np.savetxt(f, rh_array, fmt='%d')
print("NumPy array saved as 'rh_graph.txt'")

with open(rh_path, 'r') as f:
    ab = np.loadtxt(f, dtype=np.int_)
print(f"testing for shape, rh_graph is {ab.shape}")

#health
#2.0    20496
#3.0    13015
#1.0     9218
#4.0     2238
#0.0      535
#5.0      128
#Name: count, dtype: int64
#save health dict
#(45630, 2)
#NumPy array saved as 'rh_graph.txt'
#testing for shape, rh_graph is (45630, 2)

health
2.0    20496
3.0    13015
1.0     9218
4.0     2238
0.0      535
5.0      128
Name: count, dtype: int64
save health dict
(45630, 2)
NumPy array saved as 'rh_graph.txt'
testing for shape, rh_graph is (45630, 2)


In [ ]:
### kmeans

In [33]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np
from tqdm import tqdm

interaction_path = '../../../data/jeehoshin/baseline_dataset_allrecipe_sorted/'
embImage = np.load(interaction_path + 'data_image_features_caffe.npy')
embText = np.load(interaction_path + 'data_text_features_allMini.npy')

kmeans = MiniBatchKMeans(n_clusters=2000, init_size=512, batch_size=1024, random_state=2024)
print('feature_embedding,', embImage.shape)
kmeans.fit(embImage)
centers = kmeans.cluster_centers_  # [C * D]
print(centers.shape)

center_feature = {}
for c_id in range(len(centers)):
    arr = centers[c_id]
    center_feature[c_id] = arr

v_cluster_list = []
for each_embedding in tqdm(embImage):
    distances = [np.linalg.norm(each_embedding - arr) for arr in centers]
    v_cluster_list.append(list(np.argsort(distances)[:10]))

# feature_embedding, (45630, 4096)
# (2000, 4096)

feature_embedding, (45630, 4096)


/opt/conda/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:2082: RuntimeWarning: init_size=512 should be larger than n_clusters=2000. Setting it to min(3*n_clusters, n_samples)
  self._check_params_vs_input(X)


(2000, 4096)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45630/45630 [14:28<00:00, 52.53it/s]


In [34]:
v_cluster_edge = []
for iid, c_set in tqdm(zip(list(np.arange(len(embImage))), v_cluster_list)):
    c_set = c_set[:6]
    for cid in c_set:
        v_cluster_edge.append([iid, cid])
print(len(v_cluster_edge))
imageC_array = np.array(v_cluster_edge).astype(np.int_)
print(imageC_array.shape)

imageC_path = interaction_path + 'cluster/image_cluster_edge.txt'
with open(imageC_path, 'w') as f:
    np.savetxt(f, imageC_array, fmt='%d')
print("NumPy array saved as 'image_cluster_edge.txt'")

np.save(interaction_path + 'cluster/image_center.npy', centers)

# 273780
# (273780, 2)
# NumPy array saved as 'image_cluster_edge.txt'

45630it [00:00, 289306.07it/s]


273780
(273780, 2)
NumPy array saved as 'image_cluster_edge.txt'


In [35]:

kmeans = MiniBatchKMeans(n_clusters=2000, init_size=512, batch_size=1024, random_state=2024)
print('feature_embedding,', embText.shape)
kmeans.fit(embText)
centers = kmeans.cluster_centers_  # [C * D]
print(centers.shape)

center_feature = {}
for c_id in range(len(centers)):
    arr = centers[c_id]
    center_feature[c_id] = arr

t_cluster_list = []
for each_embedding in tqdm(embText):
    distances = [np.linalg.norm(each_embedding - arr) for arr in centers]
    t_cluster_list.append(list(np.argsort(distances)[:10]))

# feature_embedding, (45630, 384)
# (2000, 384)

feature_embedding, (45630, 384)


/opt/conda/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:2082: RuntimeWarning: init_size=512 should be larger than n_clusters=2000. Setting it to min(3*n_clusters, n_samples)
  self._check_params_vs_input(X)


(2000, 384)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45630/45630 [09:27<00:00, 80.38it/s]


In [36]:
t_cluster_edge = []
for iid, c_set in tqdm(zip(list(np.arange(len(embText))), t_cluster_list)):
    c_set = c_set[:6]
    for cid in c_set:
        t_cluster_edge.append([iid, cid])
print(len(t_cluster_edge))
textC_array = np.array(t_cluster_edge).astype(np.int_)
print(textC_array.shape)
textC_path = interaction_path + 'cluster/text_cluster_edge.txt'
with open(textC_path, 'w') as f:
    np.savetxt(f, textC_array, fmt='%d')
print("NumPy array saved as 'text_cluster_edge.txt'")

np.save(interaction_path + 'cluster/text_center.npy', centers)
print("finished")
# 273780
# (273780, 2)
# NumPy array saved as 'text_cluster_edge.txt'

45630it [00:02, 16624.15it/s]


273780
(273780, 2)
NumPy array saved as 'text_cluster_edge.txt'
finished
